# Agents to parse and comment an income statement

This time we will try and use agents to download, parse, and comment on an income statement.

The document is real, and it is publicly available on the internet at the time of writing.

# Setup: packages and environment variables

In [ ]:
import importlib

from dotenv import load_dotenv

if not importlib.util.find_spec("class_utils"):
    !pip install -qqq git+https://github.com/xtreamsrl/genai-for-engineers-class

load_dotenv()

In [ ]:
import autogen

# Creating agents

For this purpose, we will use the latest model, GPT-4o.

Less advanced models are less effective in writing code.

In [ ]:
llm_config = {
    "config_list": [{"model": "gpt-4.1"}],
}

# User Proxy
UserProxyAgent is a special type of agent that can be used to proxy user input to another agent or group of agents. It supports the following human input modes:
- ALWAYS: Always ask user for input.
- NEVER: Never ask user for input. In this mode, the agent will use the default response (if any) to respond to the message. Or using underlying LLM model to generate response if provided.
- AUTO: Only ask user for input when conversation is terminated by the other agent(s). Otherwise, use the default response (if any) to respond to the message. Or using underlying LLM model to generate response if provided.

We will execute the code natively without docker. This is a security flaw, and in real life it should be avoided, as the execution of arbitrary code can expose sensitive data.

In [ ]:
user_proxy = autogen.UserProxyAgent(
    name="User_proxy",
    system_message="A human admin.",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "income_statement_analysis",
        "use_docker": False,
    },
    human_input_mode="NEVER",
)

# Software Engineer, Software Engineer Critic, and Financial Analysists

Next we define our agents according to their role.

In essence, an agent is definied by its system prompt. Then, it will take part in the conversation as if it was a human being.

We will define:
- a software engineer, taked with developing the python code to perform a task,
- a software engineer critic, that will criticise the code written by the coder, and suggest improvements, and
- a financial analyst, that should read and understand the income statement and create a report about it.

Once the chat is triggered, the agents should collaborate autonomously.

In [ ]:
sw_eng = autogen.AssistantAgent(
    name="Software_Engineer",
    description="Software Engineer writing code",
    system_message="""
    You are helpful Senior Software Engineer, highly skilled in writing python code.
    When there are tasks that require writing code, such as extracting data from documents, 
    you must provide the snippets.
    In case updates are required, you must provide them.
    In case bugs are found, you must fix them quickly.
    When you extract text from a document, always print the extracted text.
    When you are asked for some data, always write some code to print it.
    Always save your code to a file named "script.py".
    Always parse full documents, not just parts of them.
    """,
    llm_config=llm_config,
)

In [ ]:
sw_eng_critic = autogen.AssistantAgent(
    name="Software_Engineer_Critic",
    description="Software Engineer evaluating code",
    system_message="""
    You are a Senior Software Engineer, highly skilled in evaluating python code.
    When some code is proposed by other agents, you must evaluate it and provide feedback.
    If you find any syntax error or logic errors, you must point them out, providing effective actions to fix them.
    If you find any issue harming the readability or efficiency of the code, you must provide feedback.
    Never write code, just suggest improvements and then ask the Software Engineer to improve the code.
    """,
    llm_config=llm_config,
)

In [ ]:
financial_analyst = autogen.AssistantAgent(
    name="Financial_Analyst",
    description="Financial Analyst evaluating income statements",
    system_message="""
    You are a highly skilled Financial Analyst, specialized in evaluating income statements.
    You have perfect knowledge of the structure of an income statement and can evaluate it effectively.
    You know how to write a brief, effective, yet precise report on the evaluation of an income statement.
    Only speak after the data has been extracted, otherwise you must ask the Software Engineer to extract the data.
    Remember, you must ask specifically to the Software Engineer.
    Once you have all the data you need, you must write a brief report on the evaluation of the income statement 
    and must must ask the Software Engineer to write and save a markdown file "report.md" with your report.
    If some data extracted from the income statement is not clear, you must ask for clarification.
    Never suggest code, just comment on the extracted data.
    Once all the tasks have been completed, you must terminate the conversation with the special 
    command "TERMINATE CONVERSATION". Use this command only when you have completed all the tasks.
    """,
    llm_config=llm_config,
    is_termination_msg=lambda msg: "terminate conversation"
    in msg.get("content").lower(),
)

In [ ]:
group_chat = autogen.GroupChat(
    agents=[user_proxy, sw_eng, sw_eng_critic, financial_analyst],
    messages=[],
    max_round=30,
    send_introductions=True,
)

manager = autogen.GroupChatManager(
    groupchat=group_chat,
    llm_config=llm_config,
)

# Run the chat
Let's trigger the grup chat and see what happens.

In [ ]:
link = "https://www.cisiaonline.it/sites/default/files/AmmTrasparente/2025/Schema-bilancio-consuntivo-CISIA-2024-e-nota-di-accompagnamento.pdf"

user_proxy.initiate_chat(
    manager,
    message=f"At this link is an income statement: {link}. "
    "Please evaluate it, considering it is written in Italian.",
)